# etherscan_api

> Endpoints for the `etherscan` api that we couldn't find extant in other packages: https://docs.etherscan.io/

**Functionality:**

`get_contract_creator`  
Obvious

`get_first_n_addresses`  
Get the first n addresses holding a token. This is currently hacky, and there is an issue with MEV bots. But working ok enough (I think) to use.

`get_last_n_transactions_for_erc20`
Not tested and can't think of use for it atm.

`get_creation_date`
Obvious

In [ ]:
#|default_exp etherscan_api

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests
import json
import time

In [ ]:
#| hide
from fastcore.test import *

In [ ]:
#| export
 
api_key = 'DWNVAVM1GPZK3PUIKM79AQ2ZZS1JUPI417'  #my API key for etherscan

In [ ]:
#| export

#Several relevant addresses for illustration purposes in the notebook (including testing etc).
pepe_address = '0x6982508145454Ce325dDbE47a25d4ec3d2311933' #Pepe address
pepe_deployer = '0xfbfEaF0DA0F2fdE5c66dF570133aE35f3eB58c9A' #Pepe deployer
bybit_address = '0xf89d7b9c864f589bbF53a82105107622B35EaA40'  #ByBit hot wallet associated with Pepe deployer....
fuckrace_address = "0x4911E8cBe156589E8516769800Ed494F34668E55"
fuckrace_deployer = "0xf627853a09E49B4D9D4A3e3913Da80a3f6d741b7"


In [ ]:
#| export

from typing import List
def get_contract_creator(contract_addresses: List[str], api_key: str) -> str:
    """
    Retrieves the contract creator and the transaction hash of the contract creation transaction.
    API docs: https://docs.etherscan.io/api-endpoints/contracts
    Args:
        contract_addresses (List[str]): A list of contract addresses to look up. Accepts up to 5.
        api_key (str): The API key for accessing the necessary web service.

    Returns:
        dict: where the keys are the contract_addresses and the values are dicts containing the creator address and the transaction hash of the contract creation transaction.
    """

    assert len(contract_addresses) <= 5, "Only up to 5 contract addresses can be looked up at a time."


    base_url = f"https://api.etherscan.io/api?module=contract&action=getcontractcreation&contractaddresses="

    for address in contract_addresses:
        base_url += f"{address},"
    
    base_url=base_url.rstrip(',')

    base_url += f"&apikey={api_key}"
            
            #&apikey={api_key}}"

    response = requests.get(base_url)
    data = json.loads(response.text)

    if data['status'] =='0':
        print("Error:", data['message'])
        return None

    data=data['result']
    data_dict={}
    for item in data:

        data_dict[item['contractAddress']] = item
    
    return data_dict


A simple test that `get_contract_creator` is working as expected:

In [ ]:
#| hide

#test
_addresses = [pepe_address,fuckrace_address]
_data = get_contract_creator(_addresses,api_key)

#Annoying wrinkle with lower case
test_eq(_data[pepe_address.lower()]['contractCreator'],pepe_deployer.lower())
test_eq(_data[fuckrace_address.lower()]['contractCreator'],fuckrace_deployer.lower())

`get_total_eth_transacted` seems to be redundant now as we can directly get porfolio info via covalent api.
#TODO: delete this later if this is in fact true.

In [ ]:
#| export

# def get_total_eth_transacted(address, api_key):
#     url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock=0&endblock=99999999&sort=asc&apikey={api_key}"
#     response = requests.get(url)
#     transactions = json.loads(response.text)['result']
    
#     total_in = 0
#     total_out = 0
#     for txn in transactions:
#         value = int(txn['value']) / (10 ** 18)  # Convert from Wei to Ether
#         if txn['to'] == address.lower():
#             total_in += value
#         elif txn['from'] == address.lower():
#             total_out += value
    
#     return total_in, total_out

#Code to handle arbitrary number of transactions, which however seems to slow things down a lot, and we possibly do not
#even need it. So, we will stick to the above code for now.

# def get_total_eth_transacted(address, api_key):
#     startblock = 0
#     endblock = 99999999
#     total_in = 0
#     total_out = 0
    
#     while True:
#         url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock={startblock}&endblock={endblock}&sort=asc&apikey={api_key}"
#         response = requests.get(url)
#         transactions = json.loads(response.text)['result']
        
#         for txn in transactions:
#             value = int(txn['value']) / (10 ** 18)  # Convert from Wei to Ether
#             if txn['to'] == address.lower():
#                 total_in += value
#             elif txn['from'] == address.lower():
#                 total_out += value

#         # If less than 10000 transactions were returned, we've got them all
#         if len(transactions) < 10000:
#             break
#         else:
#             # Use the 'blockNumber' of the last transaction as 'startblock' for the next request
#             startblock = int(transactions[-1]['blockNumber']) + 1
            
#         # Sleep for a short period to respect API rate limits
#         time.sleep(0.2)
    
#     return total_in, total_out


In [ ]:
#| export

def get_first_n_addresses(contract_address:str, n:int, api_key:str)->dict:
    """Get the first n addresses holding a contract (e.g. erc20).
       API docs: https://docs.etherscan.io/api-endpoints/accounts#get-a-list-of-erc20-token-transfer-events-by-address
        Args:
            contract_address (str): The address of the contract.
            n (int): The number of first holders' addresses to retrieve.
            api_key (str): API key to authenticate the request.

        Returns:
            dict: A dictionary containing the first n addresses holding the contract. 
                  The keys are the addresses (which is the main thing we want). Values are tuples: (tx_hash,count)
                  where tx_hash is the hash of the transaction wherein the address received the contract
                  (mostly useful for debugging/logging purposes) and count records the order.
                  So e.g. count=0 ~ means the address is the first holder of the contract. We use the ~ symbol since e.g. mev
                  bots can be excluded from etherscan page.

        NOTE: mev bot transactions seem to not show up on etherscans page, so be careful with debugging.
        TODO: possibly need a way to handle the mev bot issue.

    """
    holders_dict = {}
    count = 0
    page_num = 1
    
    while count < n:
        url = f"https://api.etherscan.io/api?module=account&action=tokentx&contractaddress={contract_address}&startblock=0&endblock=99999999&page={page_num}&offset=100&sort=asc&apikey={api_key}"
        response = requests.get(url)
        data = response.json()
        
        if 'result' not in data or not data['result']:
            # No more data to process
            break
        
        for tx in data['result']:
            address = tx['to']
            if holders_dict.get(address,None)==None:  # Avoid duplicates
                holders_dict[address] = (tx['hash'],count)
                count += 1
                if count == n:
                    return holders_dict
        
        # Move to the next page of results
        page_num += 1

    return holders_dict


Initial test / sanity check of `get_first_n_addresses`:

In [ ]:
#| hide

#test
_holders_dict = get_first_n_addresses(fuckrace_address,5,api_key)

for _address,(tx_id,count) in _holders_dict.items():

    print(f"The {count}-th holder of contract is: {_address} with tx_id={tx_id}")

    if count==0: test_eq(_address,"0xf627853a09e49b4d9d4a3e3913da80a3f6d741b7")
    if count==1: test_eq(_address,"0x4911e8cbe156589e8516769800ed494f34668e55")

#seems to work ok at least on this guy, inspecting the first few elements. Ok. 

The 0-th holder of contract is: 0xf627853a09e49b4d9d4a3e3913da80a3f6d741b7 with tx_id=0x585a4a6cacd6dae2c14e348b195df2779728e5f0d7d5fb80a68dcb4df19d68e3
The 1-th holder of contract is: 0x4911e8cbe156589e8516769800ed494f34668e55 with tx_id=0x75b329de189b34dbbb53853c9b76e18ac70f62c1ef852c77c3ae3e139ab2c38c
The 2-th holder of contract is: 0xbbd52fac458d29050128c8d62edff4a5e7cdb0a0 with tx_id=0x7e008702f0d19b4a31b8017f8b5cae71a6ae0a8bbc59660065dbd86a088dc855
The 3-th holder of contract is: 0x77c39ce537b28b1b0df41e0e6cb8c34a2d85286c with tx_id=0xe0b2a091fa01cc681be5bf6b59f16131635db7deb19414fb3915c1f27638a78e
The 4-th holder of contract is: 0x13553f6e4f6679e4f024748c068bc0c87bc90b8b with tx_id=0xf18bcaa79fad671f93d3c9a5fa0abf1570087f42e572d5d7c45be438c8240435


#NOTE on this test
We can verify that the first 3 are on the coins etherscan page. The first 2 match the first 2 on etherscan page. However, the third hash here is not the third hash on etherscan, it is the fourth. The third etherscan page hash is `0xac0dd6c9b5b49f164ec952a94ff546cee86b1cdb6a447e7fa2db8cd6fca07adf`. The address of this transaction hash is `0x4911e8cbe156589e8516769800ed494f34668e55`. We can verify that this is the address corresponding to the second transaction, so it makes sense, since we are looking for first $n$ *unique* addresses. Since this address corresponds to the second transaction we don't recount it.

The rest of the first 10 are not on the coins page. e.g. the 10th is not, but plonking it into etherscan reveals it as a `mev` transaction which explains why it doesn't show up on etherscan page.

In [ ]:
#TODO: we need more rigorous testing of `get_first_n_addresses`

Some simple sanity checks with `get_first_n_addresses` with `pepe`:

In [ ]:
#| hide

#test
_holders_dict = get_first_n_addresses(pepe_address,220,api_key)
_first_n_addresses = list(_holders_dict.keys())
_transaction_hashes = list(_holders_dict.values())
print(f'First transaction hash is: {_transaction_hashes[0]}')
#First transaction seems correct: plonking it into etherscan yields:
#ERC-20 Tokens Transferred: From Null: 0x000...000 To Pepe: Deployer For 420,690,000,000,000 ($485,938,919.44)
#which seems ok. 

print(f'Second transaction hash is {_transaction_hashes[1]}')
#Second transaction involves pepe deployer which seems reasonable

print(f'200th transaction hash is {_transaction_hashes[199]}')
#Verified that this is a valid transaction by plonking it into etherscan

First transaction hash is: ('0x2afae7763487e60b893cb57803694810e6d3d136186a6de6719921afd7ca304a', 0)
Second transaction hash is ('0xb38bba8bf8f61029a31e027c37fbc1065db784a39501d456a05cc490db5b4578', 1)
200th transaction hash is ('0x970c88c76f8c9a6dec2067fa9f5d0731c1d3fcba7aab54b279513dc53f04b982', 199)


Can't think of a strong use for the folowing function `get_last_n_transactions_for_erc20` at the moment. Just leave it here for now.

In [ ]:
#| export

def get_last_n_transactions_for_erc20(contract_address, n, api_key):
    """Fetch the last n transactions for an ERC-20 token.

    Args:
    - contract_address (str): The ERC-20 token's contract address.
    - n (int): The number of transactions to retrieve.
    - api_key (str): Your Etherscan API key.

    Returns:
    - list[dict]: A list of transaction dictionaries.
    """
    
    transactions_list = []
    page_num = 1
    
    while len(transactions_list) < n:
        url = f"https://api.etherscan.io/api?module=account&action=tokentx&contractaddress={contract_address}&page={page_num}&offset=100&sort=desc&apikey={api_key}"
        response = requests.get(url)
        data = response.json()
        
        if 'result' not in data or not data['result']:
            # No more data to process
            break
        
        transactions_list.extend(data['result'])

        # If we've collected more than 'n' transactions, truncate the list
        if len(transactions_list) > n:
            transactions_list = transactions_list[:n]
            break
        
        # Move to the next page of results
        page_num += 1

    return transactions_list

if __name__ == "__main__":
# Example usage:
    contract_address = "0x02e7f808990638e9e67e1f00313037ede2362361" #kibshi
    n = 100

    transactions = get_last_n_transactions_for_erc20(contract_address, n, api_key)
    # for tx in transactions:
    #     print(tx)


In [ ]:
#| export

from typing import Optional, Union
from datetime import datetime

def get_creation_date(api_key: str, contract_address: str, network: Optional[str] = 'mainnet') -> Union[str, None]:
    """
    Fetch the creation date of an ERC20 contract using the Etherscan API.
    
    Parameters:
    - api_key (str): The API key for Etherscan.
    - contract_address (str): The Ethereum address of the contract.
    - network (Optional[str]): The Ethereum network ('mainnet', 'ropsten', etc.). Default is 'mainnet'.
    
    Returns:
    - str: The creation date in the format 'YYYY-MM-DD HH:MM:SS' in UTC if found.
    - None: If the contract has no transactions or the API request fails.
    """
    # Determine the base URL depending on the network
    base_url = 'https://api.etherscan.io/api?'
    if network != 'mainnet':
        base_url = f'https://{network}.etherscan.io/api?'

    # Define API parameters
    params = {
        'module': 'account',
        'action': 'txlist',
        'address': contract_address,
        'startblock': 0,
        'endblock': 99999999,
        'sort': 'asc',
        'apikey': api_key
    }

    # Make API request
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = json.loads(response.text)
        if 'result' in data and len(data['result']) > 0:
            # The first transaction should be the contract creation transaction
            creation_transaction = data['result'][0]
            timestamp = int(creation_transaction['timeStamp'])
            creation_date = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
            return creation_date
        else:
            return None
    else:
        return None


In [ ]:
#| hide

#test
fuckrace_creation_date = get_creation_date(api_key, fuckrace_address)
test_eq(fuckrace_creation_date,'2023-08-03 17:53:47')
#Note that on dextools trading began ~1 day later, but visual inspection of etherscan shows that this answer is correct.
#So that is encouraging.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()